In [ ]:
!wget https://www.lamsade.dauphine.fr/~cazenave/project.zip

--2020-12-24 20:13:07--  https://www.lamsade.dauphine.fr/~cazenave/project.zip
Resolving www.lamsade.dauphine.fr (www.lamsade.dauphine.fr)... 193.48.71.250
Connecting to www.lamsade.dauphine.fr (www.lamsade.dauphine.fr)|193.48.71.250|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 275779208 (263M) [application/zip]
Saving to: ‘project.zip.4’

project.zip.4       100%[===================>] 263.00M  36.1MB/s    in 7.9s    

2020-12-24 20:13:16 (33.5 MB/s) - ‘project.zip.4’ saved [275779208/275779208]



In [ ]:
!unzip project.zip
!mv ./project/* ./

Archive:  project.zip
  inflating: project/Board.h         
  inflating: project/compileMAC.sh   
  inflating: project/compile.sh      
  inflating: project/Game.h          
  inflating: project/games.data      
  inflating: project/golois.cpp      
  inflating: project/golois.cpython-38-x86_64-linux-gnu.so  
  inflating: project/golois.py       
  inflating: project/importGolois.ipynb  
  inflating: project/README          
  inflating: project/Rzone.h         
  inflating: project/test.h5         
  inflating: project/validation.data  


In [ ]:
!pip install pybind11
!chmod 777 ./compile.sh
!./compile.sh

In file included from golois.cpp:17:0:
Board.h: In member function ‘bool Board::isCapturedLadder(int, int, Rzone*)’:
Board.h:1748:8: warning: unused variable ‘n1’ [-Wunused-variable]
    int n1 = nbLiberties (inter, liberties1, stones1, 3);
        ^~
Board.h:1769:12: warning: unused variable ‘n1’ [-Wunused-variable]
        int n1 = nbLiberties (inter, liberties1, stones1, 3);
            ^~
Board.h: In member function ‘void Board::computeLadders(int)’:
Board.h:1800:9: warning: unused variable ‘other’ [-Wunused-variable]
     int other = opponent (color);
         ^~~~~
Board.h: In member function ‘void Board::computeAllLadders(int, bool)’:
Board.h:2046:9: warning: unused variable ‘n1’ [-Wunused-variable]
     int n1 = nbLiberties (i, liberties1, stones1);
         ^~
Board.h:2082:7: warning: unused variable ‘n1’ [-Wunused-variable]
   int n1 = nbLiberties (i, liberties1, stones1);
       ^~
Board.h:2115:9: warning: unused variable ‘n1’ [-Wunused-variable]
     int n1 = nbLiberties (i

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
from tensorflow.keras import layers 
from tensorflow.keras import regularizers

import golois

planes = 21   # 21 plans 19x19 les états précédents, noires, blanc
moves = 361
N = 100000
epochs = 200
batch = 200
filters = 64
#les entrées et les sorties sont fixées mais on peut modifier le réseau
input_data = np.random.randint(2, size=(N, 19, 19, planes)) #N=nombre d'exemples, pour initialise peut-être
input_data = input_data.astype ('float32')
print(input_data.shape)#added
policy = np.random.randint(moves, size=(N,))#choisi une politique aléatoire entre 1 et 361
print(policy.shape)
policy = keras.utils.to_categorical (policy)
value = np.random.randint(2, size=(N,))
value = value.astype ('float32')
print(value.shape)

end = np.random.randint(2, size=(N, 19, 19, 2))
end = end.astype ('float32')

groups = np.zeros((N, 19, 19, 1))
groups = groups.astype ('float32')

print ("getValidation", flush = True)
golois.getValidation (input_data, policy, value, end)#récupèr 10000 exemples qui n'ont pas été appris et donnés pâr le getBatch, récupèure les entrées et sortie dans la validation , c'est pour savoir si le modèle a bien appris ou pas


input = keras.Input(shape=(19, 19, planes), name='board')
x = layers.Conv2D(filters, 1, activation='relu', padding='same')(input)# on fait une convolution avec filtre 1 juste  pour  transformer le nombre de plan pour  obtenir 64 plans 
for i in range (4):#pour model résiduel
    x1 = layers.Conv2D(filters, 3, activation='relu', padding='same')(x)
    x1 = layers.BatchNormalization()(x1)
    x1 = layers.Conv2D(filters, 3, activation='swish',padding='same')(x1)
    x = layers.add([x1,x])#x is the input
    
    x = layers.ReLU()(x)
    x=layers.Dropout(0.10)(x)
    x = layers.BatchNormalization()(x)

policy_head = layers.Conv2D(1, 1, activation='swish', padding='same', use_bias = False, kernel_regularizer=regularizers.l2(0.0001))(x)
policy_head = layers.Conv2D(filters, 3, activation='tanh', padding='same', use_bias = False, kernel_regularizer=regularizers.l2(0.0001))(policy_head)
policy_head = layers.Conv2D(1, 3, activation='relu', padding='same', use_bias = False, kernel_regularizer=regularizers.l2(0.0001))(policy_head)

#policy_head = layers.Dense(60,activation='relu')(policy_head)

#policy_head = layers.Conv2D(1, 3, activation='swish', padding='same')(policy_head)
#policy_head = layers.Conv2D(16, 3, activation='relu', padding='same', use_bias = False, kernel_regularizer=regularizers.l2(0.0001))(policy_head)
#policy_head = layers.Conv2D(8, 3, activation='relu', padding='same', use_bias = False, kernel_regularizer=regularizers.l2(0.0001))(policy_head)

policy_head = layers.Flatten()(policy_head)
policy_head = layers.Activation('softmax', name='policy')(policy_head)



value_head = layers.Conv2D(80, 3, activation='swish', padding='same')(x)
value_head=layers.Dropout(0.15)(value_head)
value_head = layers.GlobalAveragePooling2D()(value_head)
value_head = layers.Dense(30, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(value_head)
value_head=layers.Dropout(0.10)(value_head)
value_head = layers.Dense(1, activation='sigmoid', name='value', kernel_regularizer=regularizers.l2(0.0008))(value_head)




'''
value_head = layers.Conv2D(60, 3, activation='relu', padding='same')(x)
#value_head = layers.GlobalAveragePooling2D()(value_head)
value_head = layers.Dense(40, activation='swish', kernel_regularizer=regularizers.l2(0.0003))(value_head)
value_head=layers.Dropout(0.10)(value_head)
value_head = layers.Dense(30, activation='swish', kernel_regularizer=regularizers.l2(0.0003))(value_head)
value_head = layers.Dense(1, activation='sigmoid', name='value', kernel_regularizer=regularizers.l2(0.0008))(value_head)
'''
model = keras.Model(inputs=input, outputs=[policy_head, value_head])

model.summary ()

model.compile(optimizer=keras.optimizers.SGD(lr=0.008, momentum=0.95),
              loss={'policy': 'categorical_crossentropy', 'value': 'binary_crossentropy'},
              loss_weights={'policy' : 1.0, 'value' : 5.0},
              metrics={'policy': 'categorical_accuracy', 'value': 'mse'})

for i in range (1, epochs + 1):
    print ('epoch ' + str (i))
    golois.getBatch (input_data, policy, value, end, groups)#une partie jouée par catago , remplit les tenseurs avec tenseurs
    history = model.fit(input_data,
                        {'policy': policy, 'value': value}, 
                        epochs=1, batch_size=batch)
    if (i % 10 == 0):
        golois.getValidation (input_data, policy, value, end)
        val = model.evaluate (input_data,
                              [policy, value], verbose = 0, batch_size=batch)#mse sur les données de validations (su rles données qui n'ont jamais été)
        print ("val =", val)

model.save ('test.h5')


(100000, 19, 19, 21)
(100000,)
(100000,)
getValidation
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
board (InputLayer)              [(None, 19, 19, 21)] 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 19, 19, 64)   1408        board[0][0]                      
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 19, 19, 64)   36928       conv2d[0][0]                     
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 19, 19, 64)   256         conv2d_1[0][0]                   
_______________________________________

In [ ]:
from google.colab import files
files.download('test.h5')

